## TFIDF

文書中に含まれる単語の重要度を評価する手法の1つ

**tf：各文書においてその単語がどのくらい出現したのか**

tf　=　文書Aにおける単語Xの出現頻度文書　/　文書Aにおける全単語の出現頻度の和

**idf：「逆文書頻度」と呼ばれており、単語が「レア」なら高い値を、「色々な文書によく出現する単語」なら低い値を示す**

idf　=　log(　全文書数　/　単語Xを含む文書数　)

**tfidf：「その単語がよく出現するほど」「その単語がレアなほど」大きい値を示す**

tfidf　=　tf　×　idf

In [1]:
# 必要ライブラリの読み込み
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# 文章群
text_list = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

In [16]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w\\w+\\b')
tfidf = tfidf_vectorizer.fit_transform(text_list)

df = pd.DataFrame(
    data=tfidf.toarray(),
    columns=tfidf_vectorizer.get_feature_names()
)
df

,and,document,first,is,one,second,the,third,this
0,0.000000,0.469791,0.580286,0.384085,0.000000,0.000000,0.384085,0.000000,0.384085
1,0.000000,0.687624,0.000000,0.281089,0.000000,0.538648,0.281089,0.000000,0.281089
2,0.511849,0.000000,0.000000,0.267104,0.511849,0.000000,0.267104,0.511849,0.267104
3,0.000000,0.469791,0.580286,0.384085,0.000000,0.000000,0.384085,0.000000,0.384085


## TFIDFコサイン類似度推定法
### 二つの文章がどの程度に通った文章か判定する

二つの単語ベクトルの内積をとる

同じ方向を向いていれば（似通った内容）なら、1

逆の方向を向いていれば（似ていない内容）なら、-1

となる内積の性質を利用する

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
tfidf_array = tfidf.toarray()
cosine_similarity_ = cosine_similarity(tfidf_array,tfidf_array)
print(cosine_similarity_)

[[1.         0.64692568 0.30777187 1.        ]
 [0.64692568 1.         0.22523955 0.64692568]
 [0.30777187 0.22523955 1.         0.30777187]
 [1.         0.64692568 0.30777187 1.        ]]


# 実践編

In [1]:
# データの読み込み
with open('./data/sanshiro.txt', encoding='sjis') as file:
    text = file.read()

# テキスト整形
import re
text = re.split('\-{5,}',text)[2]
text = re.split('底本：',text)[0]
text = text.replace('|', '')
text = re.sub('《.+?》', '', text)
text = re.sub('［＃.+?］', '',text)
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)
text = re.sub('　', '', text)
text = re.sub(' ', '', text)

In [52]:
# テキストを「。」で区切る
text_list = text.split('。')

from janome.tokenizer import Tokenizer
def get_wakati(text):
    '''わかち書きされた文書を返す
    引数：
    　text：1文
    返り値：
    　wakati_text：わかち書きされた文書
    '''
    # インスタンス化
    t = Tokenizer()
    # 動詞と名詞のリスト
    wakati_text=''
    # 形態素解析
    res = t.tokenize(text)
    for i in res:
        wakati_text = wakati_text+i.surface+' '
    return wakati_text

wakati_text_list = [get_wakati(text) for text in text_list]

In [53]:
# TF-IDF 
tfidf_vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w\\w+\\b')
tfidf = tfidf_vectorizer.fit_transform(wakati_text_list)
data=tfidf.toarray()

In [54]:
# 各文書ごと、その文書内での最も重要な単語を抽出する
for index,i in enumerate(data):
    print("=================================")
    print(text_list[index])
    # 最も重要度の高い単語のインデックス取得
    best1_word_index = list(i).index(max(i))
    # 最も重要度の高い単語の取得
    best1_word=tfidf_vectorizer.get_feature_names()[best1_word_index]
    # 最も重要度の高い単語のインデックス取得(2番目)
    best2_word_index = list(i).index(sorted(i)[-2])
    # 最も重要度の高い単語の取得(2番目)
    best2_word=tfidf_vectorizer.get_feature_names()[best2_word_index]
    print(best1_word,best2_word)
    
    if index==10:
        break


一
　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている
うとうと うとうと
このじいさんはたしかに前の前の駅から乗ったいなか者である
いなか者 じいさん
発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬいだと思ったら背中にお灸のあとがいっぱいあったので、三四郎の記憶に残っている
ぬい ぬい
じいさんが汗をふいて、肌を入れて、女の隣に腰をかけたまでよく注意して見ていたくらいである
ふい じいさん

　女とは京都からの相乗りである
相乗り 京都
乗った時から三四郎の目についた
乗っ つい
第一色が黒い
一色 黒い
三四郎は九州から山陽線に移って、だんだん京大阪へ近づいて来るうちに、女の色が次第に白くなるのでいつのまにか故郷を遠のくような哀れを感じていた
大阪 大阪
それでこの女が車室にはいって来た時は、なんとなく異性の味方を得た心持ちがした
味方 味方
この女の色はじっさい九州色であった
九州 じっさい

　三輪田のお光さんと同じ色である
三輪田 同じ


# 実際の使い方

重要な単語がわかれば、文書が何を表す文書かということも推定できます。

単語の重要度を算出出来れば、それを文書の特徴量とみなし、クラスタリングなどに用いることもあります。

実務では、検索エンジンの構築などに利用できます。